In [1]:
#'##########################################################
#	  	      MCMC GLMM
# 	a sript for  the project
#	DEMOGRAPHIC BUFFERING CONTINUUM in PLANTS AND ANIMALS
#			 by Gabriel Santos
# 	contact by ssantos.gabriel@gmail.com
#			    05 March 2025
#' ---------------------------------------------------------
# MCMCglmms run separately so we can run it
# with intermediary data produced
#'##########################################################
set.seed(1)

rm(list=ls())

# Define necessary packages
need_pkgs <- c("tidyverse", "plotMCMC", "mcmcr", "MuMIn", "MCMCglmm","MuMIn")

exist_pckgs<-exist_pckgs <- installed.packages()[, "Package"]

if (any(!need_pkgs %in% exist_pckgs)) {   # Check for inexisting packages and install them
  install.packages(need_pkgs[!need_pkgs %in% exist_pckgs])
}




In [2]:
# load non-existing packages
lapply(need_pkgs, require, character.only = TRUE)

rm(list=ls())

Loading required package: tidyverse
── Attaching core tidyverse packages ───────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.5.1     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.1
✔ purrr     1.0.2     
── Conflicts ─────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the ]8;;http://conflicted.r-lib.org/conflicted package]8;; to force all conflicts to become errors
Loading required package: plotMCMC
Loading required package: mcmcr
Registered S3 method overwritten by 'mcmcr':
  method         from 
  as.mcmc.nlists nlist
Loading required package: MuMIn
Loading required package: MCMCglmm
Loading required package: Matrix

Attaching package: ‘Matrix’

The following objects are masked from ‘package:tidyr’:

    expand, pack, unpack

Loading required package: coda
Loading required package: ape

Attachi

Warning messages:
1: package ‘plotMCMC’ was built under R version 4.4.3 
2: package ‘mcmcr’ was built under R version 4.4.3 
3: package ‘MCMCglmm’ was built under R version 4.4.2 


In [3]:

#'===========================================================
# ----- LOAD data ready for GLMM analyses ----
# GLMM data contains:
#  - Data model
#  - subtree_Animals - Phylogenetic tree for Animals ready for analysis
#  - subtree_Animals - Phylogenetic tree for Plants ready for analysis
#'===========================================================
# USE THIS IF RUNNING IN GOOGLE COLAB
# Download data In Google colab
#dir.create(file.path("Data"), showWarnings = FALSE) # Will return warning message if folder already exists
#GLMMdata_link<-"https://github.com/Ecosantos/Demogbuff-pops/raw/refs/heads/incorporating-MCMCGlmm/Data/GLMMdata.Rdata"
#download.file(GLMMdata_link, "Data/GLMMdata.Rdata", mode = "wb")
#'===========================================================

load("Data/GLMMdata.Rdata")

data_model%>%glimpse()

#'===========================================================
# ---- Define GLMM SETTINGS ----
#'===========================================================

fixEffect<-fixEffect<-"~LHPC.1 * LHPC.2 + ClimPC.1 * ClimPC.2"
InterestingVars<-c("Survival","Growth","Shrinking","Reproduction","Clonality","Buffmx","Cumulative")

traits<-traits_glmm<- unique (grep(paste(InterestingVars,collapse="|"),
                                   colnames(data_model), value=TRUE))

#'-----------------------------------------------------------
# ---- Define priors ----
#'-----------------------------------------------------------

prior_phylo<-list(G=list(G1=list(V=1,nu=0.02)),
                  R=list(V=1,nu=0.02))

nitt=100000; #nitt=1000
burnin=1000; #burnin=100
thin=10

glmmScale<-"FALSE"

Rows: 121
Columns: 25
$ ID                   <chr> "Ephr.267_629", "Hypr.463_756", "Agrm.300_46",…
$ Reproduction_SigElas <dbl> 3.746532e-04, 6.467530e-02, 2.274530e-05, 3.71…
$ Growth_SigElas       <dbl> 7.572104e-04, 4.189061e-03, 2.648489e-03, 6.84…
$ Shrinking_SigElas    <dbl> 3.269931e-05, 1.993659e-03, 5.880106e-05, 7.21…
$ Clonality_SigElas    <dbl> 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0…
$ Survival_SigElas     <dbl> 0.0014925072, 0.1199856518, 0.0065147673, 0.06…
$ Cumulative_SigElas   <dbl> 0.0025916716, 0.1908436699, 0.0090817103, 0.07…
$ Buffmx               <dbl> 1.0004237, 1.1469247, 1.0117730, 1.0988250, 1.…
$ MatRep               <int> 4, 53, 4, 7, 3, 4, 4, 4, 4, 4, 5, 6, 3, 4, 4, …
$ LHPC.1               <dbl> 1.3246483, -1.4249240, 0.5607734, -0.9696214, …
$ LHPC.2               <dbl> 0.53785488, -0.64906960, 0.16452323, -0.340730…
$ ClimPC.1             <dbl> 0.1901417, -1.7561908, 1.8598983, 0.4375308, -…
$ ClimPC.2             <dbl> -0.28772696, 0.04956925, 

In [9]:
#'===========================================================
#	PHYLOGENETIC MCMC GLMM
#'===========================================================
MCMCglmm_phylo_plants<-NULL

#Plants
for(i in 1:length(traits)){
print(paste("Running model:",traits[i],"~",fixEffect))
MCMCglmm_phylo_plants[[i]]<-MCMCglmm(formula(paste0(traits[i], fixEffect)),
    random=~phylo,family="gaussian",
		ginverse=list(phylo=inverseA(subtree_Plants,nodes="TIPS",scale=TRUE)$Ainv),
				prior=prior_phylo,data=subset(data_model,Kingdom=="Plantae"),
   						nitt=nitt,burnin=burnin,thin=thin,singular.ok=TRUE, scale = glmmScale)

names(MCMCglmm_phylo_plants)[i]<-traits[[i]]
}

#'===========================================================
#	SIMPLE MCMC GLMM
#'===========================================================
MCMCglmm_simple_plants<-NULL

#Plants
for(i in 1:length(traits)){
print(paste("Running model:",traits[i],"~",fixEffect))
MCMCglmm_simple_plants[[i]]<-MCMCglmm(formula(paste0(traits[i], fixEffect)),
   family="gaussian",data=subset(data_model,Kingdom=="Plantae"),
   			nitt=nitt,burnin=burnin,thin=thin,singular.ok=TRUE, scale = glmmScale)

names(MCMCglmm_simple_plants)[i]<-traits[[i]]
}







[1] "Running model: Reproduction_SigElas ~ ~LHPC.1 * LHPC.2 + ClimPC.1 * ClimPC.2"

                       MCMC iteration = 0

                       MCMC iteration = 1000

                       MCMC iteration = 2000

                       MCMC iteration = 3000

                       MCMC iteration = 4000

                       MCMC iteration = 5000

                       MCMC iteration = 6000

                       MCMC iteration = 7000

                       MCMC iteration = 8000

                       MCMC iteration = 9000

                       MCMC iteration = 10000

                       MCMC iteration = 11000

                       MCMC iteration = 12000

                       MCMC iteration = 13000

                       MCMC iteration = 14000

                       MCMC iteration = 15000

                       MCMC iteration = 16000

                       MCMC iteration = 17000

                       MCMC iteration = 18000

                       MCMC iteratio

In [10]:
glmmOUT<-list(Simple_models=
       list(Plants = MCMCglmm_simple_plants),
     Phylogenetic_models=
       list(Plants = MCMCglmm_phylo_plants)
     )

saveRDS(glmmOUT,"Data/MCMCglmm_output.rds")

In [11]:
Phylo_models_plants<-lapply(MCMCglmm_phylo_plants,function(inner_list) summary(inner_list))

Simple_models_plants<-lapply(MCMCglmm_simple_plants,function(inner_list) summary(inner_list))


Phylo_models_plants_coefs<-lapply(Phylo_models_plants,function(inner_list) inner_list[[5]])
Simple_models_plants_coefs<-lapply(Simple_models_plants,function(inner_list) inner_list[[5]])

Phylo_models_plants_coefs<-lapply(Phylo_models_plants_coefs,as.data.frame)
Simple_models_plants_coefs<-lapply(Simple_models_plants_coefs,as.data.frame)


Phylo_models_df_plants<-lapply(Phylo_models_plants_coefs,rownames_to_column, var = "Statistics")%>%
  Map(cbind, Trait = names(.),Taxa="Plants", Model="Phylo", .)%>%do.call(rbind,.)

Simple_models_df_plants<-lapply(Simple_models_plants_coefs,rownames_to_column, var = "Statistics")%>%
  Map(cbind, Trait = names(.),Taxa="Plants", Model="Simple", .)%>%do.call(rbind,.)


GLMMs_df<-rbind(Phylo_models_df_plants,
                Simple_models_df_plants)


colnames(GLMMs_df)<-c("Trait","Taxa","Model","Statistics","post.mean","low95","high95","eff.samp","pMCMC")


#--------------------------------------------------------------------
# -----  SUMMARY SINTHESIS  -----  
#--------------------------------------------------------------------

GLMMs_df_summary<-GLMMs_df%>%
  mutate(sig=ifelse(pMCMC<=0.05,"Sig","Non-Sig"))%>%
  filter(Statistics!="(Intercept)")

In [21]:
GLMMs_df_summary

                                       Trait
Reproduction_SigElas.2  Reproduction_SigElas
Reproduction_SigElas.3  Reproduction_SigElas
Reproduction_SigElas.4  Reproduction_SigElas
Reproduction_SigElas.5  Reproduction_SigElas
Reproduction_SigElas.6  Reproduction_SigElas
Reproduction_SigElas.7  Reproduction_SigElas
Growth_SigElas.2              Growth_SigElas
Growth_SigElas.3              Growth_SigElas
Growth_SigElas.4              Growth_SigElas
Growth_SigElas.5              Growth_SigElas
Growth_SigElas.6              Growth_SigElas
Growth_SigElas.7              Growth_SigElas
Shrinking_SigElas.2        Shrinking_SigElas
Shrinking_SigElas.3        Shrinking_SigElas
Shrinking_SigElas.4        Shrinking_SigElas
Shrinking_SigElas.5        Shrinking_SigElas
Shrinking_SigElas.6        Shrinking_SigElas
Shrinking_SigElas.7        Shrinking_SigElas
Clonality_SigElas.2        Clonality_SigElas
Clonality_SigElas.3        Clonality_SigElas
Clonality_SigElas.4        Clonality_SigElas
Clonality_

In [17]:
GLMMs_df_summary%>%
  filter(Taxa=="Plants" & !c(Trait %in% c("Cumulative_SigElas","Buffmx")))%>%
  group_by(Trait)%>%group_split()

<list_of<
  tbl_df<
    Trait     : character
    Taxa      : character
    Model     : character
    Statistics: character
    post.mean : double
    low95     : double
    high95    : double
    eff.samp  : double
    pMCMC     : double
    sig       : character
  >
>[5]>
[[1]]
# A tibble: 12 × 10
   Trait        Taxa  Model
   <chr>        <chr> <chr>
 1 Clonality_S… Plan… Phylo
 2 Clonality_S… Plan… Phylo
 3 Clonality_S… Plan… Phylo
 4 Clonality_S… Plan… Phylo
 5 Clonality_S… Plan… Phylo
 6 Clonality_S… Plan… Phylo
 7 Clonality_S… Plan… Simp…
 8 Clonality_S… Plan… Simp…
 9 Clonality_S… Plan… Simp…
10 Clonality_S… Plan… Simp…
11 Clonality_S… Plan… Simp…
12 Clonality_S… Plan… Simp…
# ℹ 7 more variables:
#   Statistics <chr>,
#   post.mean <dbl>,
#   low95 <dbl>,
#   high95 <dbl>,
#   eff.samp <dbl>,
#   pMCMC <dbl>, …

[[2]]
# A tibble: 12 × 10
   Trait        Taxa  Model
   <chr>        <chr> <chr>
 1 Growth_SigE… Plan… Phylo
 2 Growth_SigE… Plan… Phylo
 3 Growth_SigE… Plan… Phylo
 